In [2]:
import xarray as xr
import rioxarray
import os
from datetime import datetime
import dask
from dask.delayed import delayed
from utils import get_file_paths, load_raster

def get_precipitation_data_time_series(directory, output_dir, start_time, end_time, ext = '.tif'):
    timer_start = datetime.now()
    file_paths = get_file_paths(directory, start_time, end_time)
    file_load_time = datetime.now()
    print(f"Found {len(file_paths)} files in the specified time range.")
    print(f"First file: {file_paths[0]}")
    print(f"Last file: {file_paths[-1]}")
    print(f"Time to find files: {file_load_time - timer_start}")

    # Load all raster files as a dask-backed xarray dataset
    delayed_datasets = [delayed(load_raster)(f) for f in file_paths]
    get_datasets_time = datetime.now()
    print(f"Time to create delayed tasks: {get_datasets_time - file_load_time}")
    datasets = dask.compute(*delayed_datasets)
    compute_time = datetime.now()
    print(f"Time to load files: {compute_time - get_datasets_time}")
    dataset = xr.concat(datasets, dim='time')
    concat_time = datetime.now()
    print(f"Time to concatenate files: {concat_time - compute_time}")

    # Compute mean precipitation for each time step
    mean_precipitation = dataset.mean(dim='time')
    mean_precipitation.name = 'mean_precipitation'  # Set a name for the DataArray
    print("Computed mean precipitation for each time step.")

    # Save to CSV
    output_file_path = os.path.join(output_dir, 'mean_precipitation.csv')
    mean_precipitation.to_dataframe().to_csv(output_file_path)
    print(f"Results saved to {output_file_path}")
    
directory = r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\From Megan\2min_rasters"
start_time = datetime(2022, 5, 1, 0, 0)
end_time = datetime(2022, 5, 2, 0, 0)
output_dir = r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation"
timer_start = datetime.now()
get_precipitation_data_time_series(directory, output_dir, start_time, end_time)

Found 720 files in the specified time range.
First file: Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\From Megan\2min_rasters\2min_2022\multiplied_20220501-000000.tif
Last file: Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\From Megan\2min_rasters\2min_2022\multiplied_20220502-000000.tif
Time to find files: 0:00:47.457640
Time to create delayed tasks: 0:00:00.019061
Time to load files: 0:02:13.544153
Time to concatenate files: 0:00:00.914581
Computed mean precipitation for each time step.
Results saved to Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\mean_precipitation.csv


<xarray.DataArray (band: 1, y: 91, x: 146)>
[13286 values with dtype=float32]
Coordinates:
  * band     (band) int32 1
  * y        (y) float64 40.86 40.85 40.84 40.83 ... 39.99 39.98 39.97 39.96
  * x        (x) float64 -106.5 -106.5 -106.5 -106.5 ... -105.1 -105.1 -105.1
Attributes:
    transform:      (0.00999999971424488, 0.0, -106.55000066995284, 0.0, -0.0...
    crs:            +init=epsg:4326
    res:            (0.00999999971424488, 0.009999999714204058)
    is_tiled:       0
    nodatavals:     (-3.3999999521443642e+38,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area
